# 02. Generalized linear models (GLMs) of neural responses

<div class="warning" style='padding:0.5em;background-color:#f1f1f1;border:1px solid black;width:95%'>

Lesson time: 60 m   
Contributors: Arne Mayer, Davide Spalla

---
### In this lesson you will learn:
- What GLMs are
- How they can be used to model the response to single neurons to different stimuli
- What is the difference between a purely linear, a gaussian and a poisson response model

</div>


## Introduction
---
 
In this lesson you will learn how to model the response of a neuron as a function of an external stimulus using [Generalized linear models (GLM)](https://en.wikipedia.org/wiki/Generalized_linear_model).

These concept will be particularly useful in the study of early sensory processes, where the activity of single neurons can be well carachterized as as an **input-output function** – taking a sensory stimulus (such as an pattern of photons hitting our retina, or the mechanical vibrations that we pereive as sounds) and producting a pattern of actvity.   

This phenomenon can be studied with many different tools: tuning curves (ref), ... to name a few. 
In this lesson we will focus on GLM, a class of simple but powerful models that can be use to describe how a the firing rate of a neuron depends on the value of sensory stimuli.





## Stimulus-response functions
---
A major challenge with characterizing neural responses to sensory stimuli is that the possible space of inputs is very large, possibly infinite. An example from the auditory field might help to illustrate the problem. Imagine that you have a piano with 88 keys and you want to understand how a neuron in the auditory cortex is processing sounds (here: the tones that are generated when pressing one or more piano keys). All sensory neurons are more or less nonlinear. Thus the neural response to the combination of keys C1 and E1 is not the same as the sum of C1 or E1 when presented alone. As a consequence, probing the neuron with all possible key combinations would require $2^{88} - 1$ "stimuli". Even if we fix the duration of each stimulus to 1 second and use the same sound level this would exceed the lifetime of any species used to study these principles in the laboratory. Thus, we have to do something smarter.


**Generalized linear models (GLMs)**


An alternative approach is to use a statistical model that seeks to characterize response to complex stimuli. 
This allows us to restrict the shape that the neural response can take to a much smaller hypothesis space, using some assumption on the kind of function that the neuron implements

GLMs are a powerful and popular set of such statistical models, that hypothesizes that the neural responses is a result of two stages: a linear filter of the input and a static non-linearity.

**1. Linear Stage**

The **linear stage** consists of one or more **linear filters** represented by the vectors $\mathbf{k}_1, \mathbf{k}_2, ...$ that describes how a neuron is integrating stimulus features (defined below). In this leson we will focus on models with a single filter $\mathbf{k}$. The linear filters are linked to the concept of a **receptive field (RF)** and indeed both terms are used interchangeably. Broadly speaking, the RF is a portion of **sensory space** that can elicit neuronal responses when stimulated. For example, a neuron in the visual cortex that elicits an action potential when an object appears in the upper left visual field is said to have a RF at that location. For a visual neuron, the sensory space is typically 2-dimensional (e.g., a x/y positions of pixels on a display used in laboratory settings) or 3-dimensional (e.g., pixel x/y positions that change over time). The precise geometry of the sensory space does often not matter. Instead, we treat every "element" (e.g., a pixel of an image) of this space as a single dimension in a usually high-dimensional "feature" space (see above image) and write the stimulus as a vector 

$$
\mathbf{s} = (s_1, s_2, s_3, ..., s_D)^T.
$$

The output of the linear stage is simply the dot product of the linear filter and the stimulus:

$$
x = \mathbf{k}^T \mathbf{s} = \sum_{i=1}^D k_i s_i.
$$

Thus, the entries in the vector $\mathbf{k}$ can be interpreted as the weights that a neuron gives to different stimulus features. For the visual neuron mentioned above, the weights would be non-zero for a pixel values in the upper left visual field, and zero otherwise. It is exactly this **simple interpretation of model parameters in the original stimulus space** that makes GLMs so appealing and is one or the main reasons why it is extremely popular in neural coding. 

**2. Nonlinear Stage**

The nonlinear stage that transforms the linearly filtered stimulus $x$ into a spike rate using a static, memoryless nonlinearity $f$. The output of the nonlinear stage for a single filter is given by $f(\mathbf{k}^T\mathbf{s})$. For dynamically fluctuating stimuli, the time-varying output of the LN model can simply be described by $f(\mathbf{k}^T\mathbf{s}_t)$ where $\mathbf{s}_t$ is the stimulus at discrete time step $t$.

The type of non-linearity uses in the non-linear stage defines the different kind of GLM
In this lesson, we will discuss the **linear-Gaussian model** and the **linear-nonlinear Poisson model**.


### Linear-Gaussian models
---

In the simplest case the response is assumed to be modeled directly by the output of a single filter, possibly with a constant offset response:

$$
    r_t = \mathbf{k}^T \mathbf{s}_t + r_0 + \mathcal{N}(0, \sigma^2)
$$

where response variability (which inevatibly arises in neural data) is Gaussian-distributed with constant variance $\sigma^2$ around the filter output. The constant offset term $r_0$ can be conveniently absorbed into the RF vector $\mathbf{k}$ by setting an additional dimension in the stimulus vector $\mathbf{s}_t$ to 1 at all times, so that the offset becomes the coefficient associated with this added dimension. Thus, we will typically omit explicit reference to (and notation of) the offset term.

Given a stimulus and a measured response, estimated filter weights $\hat{\mathbf{k}}$ can be obtained by minimizing the squared difference between the model output and the measured data:

\begin{equation}
\hat{\mathbf{k}} = \underset{k}{\mathrm{argmin}} \sum_{t=1}^{N} || r_t - \mathbf{k}^T \mathbf{s}_t||^2 = (\mathbf{S}^T\mathbf{S})^{-1}\mathbf{S}^T\mathbf{r}
\end{equation}

where $\mathbf{S}$ is the stimulus design matrix formed by collecting the stimulus vectors as rows, $\mathbf{S} = (\mathbf{s}_1, \mathbf{s}_2, ..., \mathbf{s}_N)^T$ and $\mathbf{r}$ is a column vector of corresponding measured responses. The matrix product $\mathbf{S}^T \mathbf{r}$ gives the sum of all stimuli that evoked spikes (with stimuli evoking multiple spikes repeated for each spike in the bin); if divided by the total number of spikes this would be the spike-triggered average (STA) stimulus. The term $\mathbf{S}^T \mathbf{S}$ is the stimulus auto-correlation matrix; pre-multiplying by its inverse removes any structure in the STA that might arise from correlations between different stimulus inputs, leaving an estimate of the RF filter. 

More generally, the above equations corresponds to the maximum likelihood estimator (MLE) for a model in which response variability is Gaussian-distributed with constant variance around the filter output $x_t$ (for details see the review paper mentioned above).

Before we start with the linear model, we define a number of helper functions to that are generally useful for data generation throughout the assignment:

In [35]:
def create_receptive_field(size=(15, 15),
                           mu=(8, 8),
                           sigma=(4, 4),
                           angle=45,
                           frequency=.085,
                           phase=0.):
    """2D Gabor-like receptive field
    
    Parameters
    ----------
    size: tuple
      (width, height) of the 2D RF
    mu: tuple
      (x, y) center coordinates of the Gaussian envelope
    sigma: tuple
      standard deviation of the Gaussian envelope
    angle: float
      Angle of the grating (in degrees)
    frequency: float
      Spatial grating frequency
    phase: float
      Spatial phase (in degrees)
    """

    xx, yy = np.meshgrid(1. + np.arange(size[0]),
                         1. + np.arange(size[1]))

    # Gaussian envelope
    G = np.exp(- np.power(xx - mu[0], 2) / (2. * sigma[0])
               - np.power(yy - mu[1], 2) / (2. * sigma[1]))

    # spatial modulation
    phi = np.deg2rad(angle)
    xxr = xx * np.cos(phi)
    yyr = yy * np.sin(phi)
    xyr = (xxr + yyr) * 2. * np.pi * 2. * frequency
    S = np.cos(xyr + phase)

    K = G * S
    K /= np.amax(np.abs(K))

    return K


def create_gaussian_stimuli(n_bins, n_dim,
                            std_dev=1.,
                            append_ones=True):
    """Gausssian white noise stimulus matrix
    
    Parameter
    ---------
    n_bins: int
        Number of time steps
    n_dim: int
        RF dimensionality (= stimulus space dimensionality)
    std_dev: float
        Standard deviation of the Gaussian white noise
    append_ones: bool
        Append a column of ones to the stimulus matrix
    """

    S = std_dev * np.random.randn(n_bins, n_dim)

    if append_ones:
        # append a row with ones for fitting the offset term
        S = np.hstack((S, np.ones((n_bins, 1))))

    return S

def f_identity(x):
    # identity function used in a truly linear model

    return x


def f_threshold_quadratic(x):
    # threshold-quadratic nonlinearity

    y = np.copy(x)
    y[y < 0] = 0

    return y**2


def f_quadratic(x):
    # fully quadratic nonliearity

    return x**2


def generate_data_linear(rf_size=(15, 15),
                         f_nonlin=f_identity,
                         duration=10.,
                         dt=.1,
                         offset=2.,
                         noise_variance=4):
    """create model, stimulus set and and simulate neural response
    
    The model consists of the two stages:
    1. A 2-dimensional RF that is used to filter 2D Gaussian white noise stimuli
    2. A threshold-linear nonlinearity (as neural firing rates cannot be negative)

    Note: by using a large offset term (r_0) the model can be made linear as the product k x s will be 
    positive. However, for the chosen stimulus class (Gaussian white noise) the linear estimator provides 
    an unbiased estimate even in the presence of a nonlinearity (Bussgang Res. Lab. Elec. (1952), Paninski 
    Network (2003)).

    Parameters
    ----------
    duration: float
        Length of the data sequence in seconds
    f_nonlin: function-like
        The (potentially nonlinear) function that transforms k x s into a neural response (default: identity)
    dt: float
        Bin width (= time resolution) in seconds
    offset: float
        Offset term (see "r_0" in the description above)
    noise_variance: float
        Response noise variance
    """

    assert duration > 0 and dt > 0 and noise_variance >= 0

    # get number of non-overlapping time bins
    n_bins = round(duration / float(dt))

    # Gabor-like receptive field
    K = create_receptive_field(size=rf_size)  # 2D RF -> matrix

    # append entry to "true" RF (ravel() vectorizes the 2D RF matrix into a 1D vector)
    k_with_offset = np.append(K.ravel(), offset)

    # generate Gaussian stimuli
    n_dim = K.size  # dimensionality of "vectorized" RF (= dim. of stimuli); the same as K.shape[0]*K.shape[1]
    S = create_gaussian_stimuli(n_bins, n_dim,
                                append_ones=True)

    # 1. linear stage
    ks = np.dot(k_with_offset, S.T)

    # 2. nonlinear stage (for a truly linear model: f -> identity function)
    rate = f_nonlin(ks)

    # add Gaussian noise centered around the "true" rate for each bin
    rate = rate + np.sqrt(noise_variance) * np.random.randn(n_bins)

    return K, S, ks, rate


Let's plot model parameters together with the generated neural response

In [1]:
# simulate data
duration = 10.
dt = .1
f_nonlin = f_identity  # for a linear model
K_true, S, ks, rate = generate_data_linear(f_nonlin=f_nonlin,
                                           duration=duration,
                                           dt=dt)

# show example stimulus, receptive field, and static nonlinearity
fig, axes = plt.subplots(nrows=1, ncols=3)

ax = axes[0]
ax.set_title(r'Example Gaussian white noise stimulus $\mathbf{s}$')
ax.imshow(S[0, :-1].reshape(K_true.shape))
ax.set_xlabel('x')
ax.set_ylabel('y')

ax = axes[1]
ax.set_title(r'Receptive field $\mathbf{k}$')
ax.imshow(K_true, vmin=-1, vmax=1)
ax.set_xlabel('x')
ax.set_ylabel('y')

ax = axes[2]
ax.set_title('Linear transform (+ Gaussian noise)')
xx = np.linspace(ks.min(), ks.max(), 100)
ax.plot(xx, f_nonlin(xx), 'r-')
ax.scatter(ks, rate, s=10, c=[3*[.1]])
ax.set_xlabel(r'$\mathbf{k}^T \mathbf{s}$')
ax.set_ylabel(r'Rate $r$')

fig.tight_layout()

# plot spike rate together with spikes
fig, ax = plt.subplots()

n_bins = len(rate)
t = (.5 + np.arange(n_bins)) * dt # show bin centers
ax.plot(t, ks, '-',
        color='tab:blue',
       label=r'$\mathbf{k}^T\mathbf{s}$')
ax.plot(t, rate, '-',
       color='tab:orange',
       label=r'Rate $r$')

ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Rate $r$')
ax.legend(loc=(.05, 1.05),
         fontsize=8).get_frame().set_visible(0)

fig.tight_layout()

NameError: name 'f_identity' is not defined

### Linear-nonlinear Poisson models
---

For spike-train responses, a natural first assumption is that spike times are influenced only by the stimulus, and are otherwise entirely independent of one another. This assumption requires that the distribution of spike times be governed by a **Poisson (point) process conditioned on the stimulus**, defined by an **instantaneous rate function**

$$
    \lambda_t = f(\mathbf{k}^T \mathbf{s}_t)
$$ 

(often also called intensity function). In turn, this means that the distributions of counts within response time bins of size $\Delta$ must follow a Poisson distribution

\begin{equation}
  P(r_t | \mathbf{s}_t, \mathbf{k}) = \frac{(\lambda_t\Delta)^{r_t}}{r_t!} e^{-\lambda_t\Delta}
\end{equation}

where $\lambda_t\Delta$ is the expected number of spikes in a small unit of time $\Delta$ and $\lambda_t$ is the rate (or intensity) of the Poisson process. As for the linear-Gaussian model, a constant offset term $r_0$ can be conveniently absorbed into the RF vector $\mathbf{k}$ by setting an additional dimension in the stimulus vector $\mathbf{s}_t$ to 1 at all times.

If $f$ is assumed to be monotonic and fixed (rather than being defined by parameters that must be fit along with the RF) then the above equation describes an instance of a **generalized linear model (GLM)** (Nelder and Wedderburn, 1972), a widely-studied class of regression models. Many common choices of $f$ result in a likelihood which is a concave function (Paninski, 2004), guaranteeing the existence of a **single optimum** that can easily be found by standard optimization techniques such as gradient ascent or Newton's method. The GLM formulation can also be extended to non-Poisson processes, by including probabilistic interactions between spikes in different bins that may be often reminiscent of cellular biophysical processes. However, here we will focus on a Poisson GLM without spike bin interactions.


In [ ]:
def generate_inhomogeneous_poisson_spikes(lamda, dt):

    n_bins = lamda.shape[0]
    bins = np.arange(n_bins+1)*dt

    # generate Poisson distributed numbers for all bins with the max. intensity (lamda_max)
    lamda_max = np.max(lamda)
    poisson_numbers = np.random.poisson(lamda_max, size=n_bins)

    # throw away numbers depending on the actual intensity ("thinning")
    spike_times = []
    prob = lamda / lamda_max
    for i in range(n_bins):
        
        # number of spikes to keep in this bin
        n = np.sum(np.random.rand(poisson_numbers[i]) < prob[i])
        n_s = int(round(n * dt))

        # generate random spike times in this bin
        ts = bins[i] + np.random.rand(n_s)*dt

        spike_times.extend(ts)

    return np.asarray(spike_times)


def generate_data_poisson(rf_size=(15, 15),
                          duration=10.,
                          dt=.1,  # delta in equation
                          offset=0.,
                          spike_rate=5.  # average spike rate
                          ):

    assert duration > 0 and dt > 0

    # get number of non-overlapping time bins
    n_bins = round(duration / float(dt))

    # Gabor-like receptive field
    K = create_receptive_field(size=rf_size)  # 2D RF -> matrix

    # generate Gaussian stimuli
    n_dim = K.size  # dimensionality of "vectorized" RF (= dim. of stimuli); the same as K.shape[0]*K.shape[1]
    S = create_gaussian_stimuli(n_bins, n_dim,
                                std_dev=.75,  # gives a more realistic spike rate (exp() nonlinearity can produce very large values)
                                append_ones=True)

    # append entry to "true" RF (ravel() vectorizes the 2D RF matrix)
    k_with_offset = np.append(K.ravel(), offset)

    # 1. linear stage
    ks = np.dot(k_with_offset, S.T)

    # 2. nonlinear stage
    lamda = np.exp(ks)

    # lamda * dt is the number of spikes in the different bins (but keep in mind that the Poisson process
    # is a stochastic process so the actual number will differ for every draw). Thus, the sum of the product 
    # across all bins gives the expected number of spikes for the whole draw.
    expected_rate = np.sum(lamda*dt) / duration
    lamda *= (spike_rate / expected_rate)

    # generate spike times using an inhomogeneous Poisson process
    spike_times = generate_inhomogeneous_poisson_spikes(lamda, dt)

    # compute spike counts in the different time bins
    spike_counts = np.histogram(spike_times,
                                bins=np.arange(n_bins+1)*dt)[0]

    print("average spike rate: %0.2f spikes per second" % (len(spike_times) / duration))

    return K, S, ks, lamda, spike_times, spike_counts


# show model parameters
duration = 10.
dt = .1
K_true, S, ks, lamda, spike_times, spike_counts = generate_data_poisson(duration=duration,
                                                                        dt=dt)


# show example stimulus, receptive field, and static nonlinearity
fig, axes = subplots(nrows=1, ncols=3)

ax = axes[0]
ax.set_title(r'Example Gaussian white noise stimulus $\mathbf{s}$')
ax.imshow(S[0, :-1].reshape(K_true.shape))
ax.set_xlabel('x')
ax.set_ylabel('y')

ax = axes[1]
ax.set_title(r'Receptive field $\mathbf{k}$')
ax.imshow(K_true, vmin=-1, vmax=1)
ax.set_xlabel('x')
ax.set_ylabel('y')

ax = axes[2]
ax.set_title('Static nonlinearity (exponential)')
ax.scatter(ks, lamda, s=10, c=[3*[.1]])
ax.set_xlabel(r'$\mathbf{k}^T \mathbf{s}$')
ax.set_ylabel(r'f($\mathbf{k}^T \mathbf{s})$')


fig.tight_layout()

# Show response
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True)

n_bins = lamda.shape[0]
t = np.arange(n_bins) * dt

ax1.step(t, ks, where='post')
ax1.set_ylabel(r'$\mathbf{k}^T \mathbf{s}$')

ax2.step(t, lamda, where='post')
ax2.set_ylabel(r'$\lambda$')

ax3.vlines(spike_times, 0, 1)
ax3.set_ylabel('1 = spike')

ax4.step(t, spike_counts, where='post')
ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Spike count')

fig.tight_layout()


The log-likelihood function and its derivative w.r.t $\mathbf{k}$ for a LNP model with exponential nonlinearity $f(\mathbf{k}^T \mathbf{s}) \equiv e^{\mathbf{k}^T \mathbf{s}}$ can be computed as follows: 
  
Log-likelihood for a single observation (at time $t$):

\begin{equation}
    \log P(r_t | \lambda_t) = r_t\log \lambda_t + r_t\log \Delta - \log r_t! - \lambda_t \Delta.
\end{equation}

Thus the log-likelihood for the whole spike sequence is given by

\begin{equation}
    \log P(R | \lambda) = \sum_t r_t\log \lambda_t + \sum_t r_t\log \Delta - \sum_t \log r_t! - \sum_t \lambda_t \Delta.
\end{equation}

The derivative w.r.t to $\mathbf{k}$ is

\begin{align}
    \frac{\partial{\log P}}{\partial{\mathbf{k}}} &= \sum_t \frac{\partial{r_t \log \lambda_t}}{\partial{\mathbf{k}}} + \sum_t \frac{\partial{r_t \log \Delta}}{\partial{\mathbf{k}}} - \sum_t \frac{\partial{r_t \log r_t!}}{\partial{\mathbf{k}}} - \Delta \sum_t \frac{\partial{\lambda_t}}{\partial{\mathbf{k}}} \\
    &= \sum_t \frac{\partial{r_t \log \lambda_t}}{\partial{\mathbf{k}}} - \Delta \sum_t \frac{\partial{\lambda_t}}{\partial{\mathbf{k}}}
\end{align}

and with $\frac{\partial{\lambda_t}}{\partial{\mathbf{k}}} = \mathbf{s}\lambda_t$

\begin{equation}
\frac{\partial{\log P}}{\partial{\mathbf{k}}} = \sum_t r_t \mathbf{s}_t - \Delta \sum_t \mathbf{s}_t \lambda_t
\end{equation}

## REAL DATA EXAMPLE 
- head direction cells? V1 gabor?

<div class="warning" style='padding:0.5em; background-color:#f1f1f1;border:1px solid black;width:95%'>

### Key points 

- GLMs are statistical models of neural responses
- They consist of a linear filter and a static non-linearity
- GLMs receptive fields shows to which part of the stimulus space a neuron responds preferentially

<div class="warning" style='padding:0.5em; background-color:#f1f1f1;border:1px solid black;width:95%'>

### References and resources

**Books & papers**
* A review on stimulus-response analysis: https://www.frontiersin.org/articles/10.3389/fnsys.2016.00109/full
* A review on receptive-fields identification: https://www.annualreviews.org/doi/10.1146/annurev-neuro-062012-170253 

**Websites & blogposts**


**Software**
* statsmodels offers an API for GLMs in python: https://www.statsmodels.org/stable/glm.html

## Exercises
You can find the exercises for this lessons in [02-exercises.ipynb](02-GLMs/02-exercises.ipynb)